# 전이학습

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# 이미지 데이터셋, 전처리, 전이학습 모듈
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights

from torchinfo import summary
from torchmetrics.functional.classification import multiclass_accuracy

In [10]:
# 데이터 준비
img_dir = '../data/catrabbit/test-images'

# Resnet 전처리
preprocessing = transforms.Compose([
    transforms.Resize(size = 256, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop(size = 224), # 224로 잘라달라
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

# 이미지 데이터 셋 생성
imgDS = ImageFolder(root = img_dir, transform = preprocessing)
print(imgDS.classes, imgDS.targets)

['cat', 'rabbit'] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [15]:
imgDL = DataLoader(imgDS, batch_size =3, shuffle = True, drop_last=True)
for img, target in imgDL:
    print(img.shape, target)

torch.Size([3, 3, 224, 224]) tensor([0, 1, 1])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 1, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 1, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])


In [16]:
# 모델 설계 및 설정

# 사전학습된 모델 인스턴스 생성
res_model = resnet18(weights = ResNet18_Weights.DEFAULT)

# 전결합층 변경
# in_features : FeatureMap에서 받은 피쳐수, out_features : 출력/분류 클래스 수
res_model.fc = nn.Linear(in_features=512, out_features=1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\kdp/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
52.5%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100.0%


In [18]:
summary(res_model, input_size=(3, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [3, 1]                    --
├─Conv2d: 1-1                            [3, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [3, 64, 112, 112]         128
├─ReLU: 1-3                              [3, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [3, 64, 56, 56]           --
├─Sequential: 1-5                        [3, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [3, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [3, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [3, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [3, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [3, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [3, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [3, 64, 56, 56]           --
│

In [24]:
# Resnet18 Feature Module 파라미터 requires_grad = True => False 비활성화
for name, param in res_model.named_parameters():
    print(name, param.requires_grad)  # 변경 전
    param.requires_grad = False
    print(param.requires_grad)  # 변경 후
print('='*40)
# Resnet18 Full Connected Module 파라미터 requires_grad = False => True 활성화
for name, param in res_model.fc.named_parameters():
    print(name, param.requires_grad)  # 변경 전
    param.requires_grad = False
    print(param.requires_grad)  # 변경 후

conv1.weight False
False
bn1.weight False
False
bn1.bias False
False
layer1.0.conv1.weight False
False
layer1.0.bn1.weight False
False
layer1.0.bn1.bias False
False
layer1.0.conv2.weight False
False
layer1.0.bn2.weight False
False
layer1.0.bn2.bias False
False
layer1.1.conv1.weight False
False
layer1.1.bn1.weight False
False
layer1.1.bn1.bias False
False
layer1.1.conv2.weight False
False
layer1.1.bn2.weight False
False
layer1.1.bn2.bias False
False
layer2.0.conv1.weight False
False
layer2.0.bn1.weight False
False
layer2.0.bn1.bias False
False
layer2.0.conv2.weight False
False
layer2.0.bn2.weight False
False
layer2.0.bn2.bias False
False
layer2.0.downsample.0.weight False
False
layer2.0.downsample.1.weight False
False
layer2.0.downsample.1.bias False
False
layer2.1.conv1.weight False
False
layer2.1.bn1.weight False
False
layer2.1.bn1.bias False
False
layer2.1.conv2.weight False
False
layer2.1.bn2.weight False
False
layer2.1.bn2.bias False
False
layer3.0.conv1.weight False
False
layer3.0

In [25]:
# 학습 준비
optimizer = optim.Adam(res_model.fc.parameters())
loss_fn = nn.BCELoss()
EPOCHS = 3